<a href="https://colab.research.google.com/github/Anay-25/Merton_Model/blob/main/Merton_calculator_equity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display

# Function for Merton Model
#B is debt
#E0 is asset value and ae is asset volatility
# T is time to maturity and r is rate of interest
def merton_model(B, E0, ae, T, r):
    arr = []

    def equations(p):
        av, V0 = p
        d1 = (np.log(V0 / B) + (r + (av ** 2) / 2) * T) / (av * np.sqrt(T))
        d2 = (np.log(V0 / B) + (r - (av ** 2) / 2) * T) / (av * np.sqrt(T))

        eq1 = V0 * norm.cdf(d1) - B * np.exp(-r * T) * norm.cdf(d2) - E0
        eq2 = norm.cdf(d1) * av * V0 - ae * E0
        return [eq1, eq2]

    guess_av = ae - 0.01
    guess_V0 = 2 * B

    av, V0 = fsolve(equations, (guess_av, guess_V0))

    # av is asset volatility, V0 is initial asset value
    d1 = (np.log(V0 / B) + (r + (av * av) / 2) * T) / (av * np.sqrt(T))
    d2 = d1 - (av * np.sqrt(T))

    Pd = norm.cdf(-d2)
    arr.append(Pd)

    L = (B * np.exp(-r * T)) / V0
    credit_spread = -np.log(((norm.cdf(d2) + norm.cdf(-d1)) / L) / T)
    arr.append(credit_spread)

    y = credit_spread + r
    market_value = V0 - E0
    promised = B * np.exp(-r * T)
    loss = ((promised - market_value) / promised) * 100
    arr.append(loss)

    return arr

d_s = widgets.FloatSlider(min=0, max=50000000, step=500000, description='Debt:')
e_s = widgets.FloatSlider(min=0, max=50000000, step=500000, description='Equity:')
v_s = widgets.FloatSlider(min=0, max=1, step=0.005, description='Volatility:')
m_s = widgets.FloatSlider(min=0, max=10, step=0.1, description='Maturity:')
r_s = widgets.FloatSlider(min=0, max=1, step=0.005, description='Interest:')

p_b = widgets.Button(description='Probability')
c_b = widgets.Button(description='Credit Spread')
l_b = widgets.Button(description='Loss')

o_p = widgets.Output()
o_c = widgets.Output()
o_l = widgets.Output()

def on_button_click(b, metric, output):
    with output:
        arr = merton_model(
            d_s.value, e_s.value, v_s.value, m_s.value, r_s.value
        )
        print(f"{metric}: {arr[metric]:.4f}")

p_b.on_click(lambda b: on_button_click(b, 0, o_p))
c_b.on_click(lambda b: on_button_click(b, 1, o_c))
l_b.on_click(lambda b: on_button_click(b, 2, o_l))

i_w = widgets.VBox([
    widgets.HBox([d_s, e_s]),
    widgets.HBox([v_s, m_s, r_s]),
])

b_w = widgets.HBox([p_b, c_b, l_b])

display(i_w, b_w, o_p, o_c, o_l)


Output()

Output()

Output()

In [ ]:
# For ITC
# Bond_price = 400
# Equity_volatility = 1.25%
# Rate of interest= 6.5%
# Equity_value= 445
# Maturity= 0.25

# Probability of default calculated is aprroximately zero
# Rating given by ICRA is A1+
